In [55]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd
from bpemb import BPEmb

from fastai.callbacks import *
from fastai.datasets import *
from fastai.imports import nn, torch
from fastai.metrics import *
from fastai.text import *
from fastai.text.data import TextLMDataBunch
from fastai.text.transform import BaseTokenizer, Tokenizer, Vocab
from fastai.train import *
import fastai

import news_utils.fastai
import news_utils.clean.german
torch.cuda.set_device(0)

In [2]:
fastai.__version__

'1.0.42'

In [3]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

In [59]:
len(bpemb_de.words)

25000

In [60]:
itos = dict(enumerate(bpemb_de.words + ['xxpad']))
voc = Vocab(itos)

In [ ]:
df_all = pd.read_pickle('/mnt/data/group07/johannes/ompc/unla.pkl')
df_all['text_cat'] = df_all.apply(lambda x: (x['Headline'] if not x['Headline'] is None else '') + ' ' + (x['Body'] if not x['Body'] is None else '') + ' xxp ' + ('xxa' if pd.isna(x['ID_Parent_Post']) else 'xxb') , axis=1)

df_all['text_ids'] = df_all['text_cat'].apply(lambda x: bpemb_de.encode_ids_with_bos_eos(news_utils.clean.german.clean_german(x)))

df_all_train = df_all[df_all['ID_Article'] < 11500]
df_all_val = df_all[df_all['ID_Article'] >= 11500]


In [ ]:
data_lm_ft = TextLMDataBunch.from_ids(device='cuda:0', bs=128, path='/mnt/data/group07/johannes/ompc/lmexp',vocab=voc, train_ids=df_all_train['text_ids'], valid_ids=df_all_val['text_ids'])

In [ ]:
data_lm_ft.save('whatever')

In [4]:
data_lm_ft = TextLMDataBunch.load(path='/mnt/data/group07/johannes/ompc/lmexp', cache_name='whatever')

In [10]:
fold = '1'
cat = 'ArgumentsUsed'
# cat = 'PersonalStories'
model_id = '2019_ 4_01_00_11_32_066215' 
exp_path = '/mnt/data/group07/johannes/ompc/exp/' + cat + '_' + fold

In [11]:
# data_lm = TextLMDataBunch.load(Path('/mnt/data/group07/johannes/germanlm/exp_1'))

In [12]:
# data_lm.vocab = voc

In [13]:
# shutil.copy('/mnt/data/group07/johannes/ompc/exp/unlab/models/enc1.pth', '/mnt/data/group07/johannes/ompc/exp/' + cat + '_' + fold +'/models/enc1.pth')

In [14]:
train_df = pd.read_pickle(Path('/mnt/data/group07/johannes/ompc/data_ann')/cat/fold/'train.pkl')
test_df = pd.read_pickle(Path('/mnt/data/group07/johannes/ompc/data_ann')/cat/fold/'test.pkl')



# train_df = train_df.dropna(subset=['Body'])
# test_df = test_df.dropna(subset=['Body'])

train_df['text_cat'] = train_df.apply(lambda x: (x['Headline'] if not x['Headline'] is None else '') + ' ' + (x['Body'] if not x['Body'] is None else '') + ' xxp ' + ('xxa' if pd.isna(x['ID_Parent_Post']) else 'xxb') , axis=1)

test_df['text_cat'] = test_df.apply(lambda x: (x['Headline'] if not x['Headline'] is None else '') + ' ' + (x['Body'] if not x['Body'] is None else '') + ' xxp ' + ('xxa' if pd.isna(x['ID_Parent_Post']) else 'xxb') , axis=1)

train_df['text_ids'] = train_df['text_cat'].apply(lambda x: bpemb_de.encode_ids_with_bos_eos(news_utils.clean.german.clean_german(x)))

test_df['text_ids'] = test_df['text_cat'].apply(lambda x: bpemb_de.encode_ids_with_bos_eos(news_utils.clean.german.clean_german(x)))

data = TextClasDataBunch.from_ids(pad_idx=25000, bs=64,path=exp_path, vocab=data_lm_ft.vocab, classes=[0, 1], train_lbls=train_df['Value'], valid_lbls=test_df['Value'], train_ids=train_df['text_ids'], valid_ids=test_df['text_ids'])

In [15]:
# train_df['Value'] = train_df['Value'].apply(lambda x: 1 if x == 0 else 0)
train_df['Value'].value_counts()

In [16]:
# data_lm = TextLMDataBunch.from_ids(bs=64,path='/mnt/data/group07/johannes/ompc/lmexp', vocab=voc, train_ids=train_df['text_ids'], valid_ids=test_df['text_ids'])

In [17]:
learn_lm = language_model_learner(data_lm_ft).load(Path('/mnt/data/group07/johannes/ompc/lmexp/models/' + model_id))

In [18]:
learn_lm.save_encoder('enc5')

In [19]:
# data.save()

In [51]:
shutil.copy('/mnt/data/group07/johannes/ompc/lmexp/models/enc5.pth', '/mnt/data/group07/johannes/ompc/exp/' + cat + '_' + fold +'/models/enc5.pth')

learn = text_classifier_learner(data, drop_mult=0.8)
learn.load_encoder('enc5')

hidden:  0.24
input:  0.32000000000000006
embed:  0.04000000000000001
weight:  0.4


In [45]:
import sklearn

In [48]:
sklearn.utils.class_weight.compute_class_weight('balanced', [0, 1], train_df['Value'])

array([0.698146, 1.761697])

In [52]:
learn.loss_func = torch.nn.CrossEntropyLoss(torch.FloatTensor(sklearn.utils.class_weight.compute_class_weight('balanced', [0, 1], train_df['Value'])).cuda())

In [ ]:
lr = 0.001
lrs = [lr / (2.6 ** (4 - x)) for x in range(4)] + [lr]
# learn.metrics += [KappaScore()]

learn.metrics += [KappaScore(), news_utils.fastai.F1Bin(), news_utils.fastai.PrecBin(), news_utils.fastai.RecaBin()]

#         learn.callbacks += [
#             SaveModelCallback(learn, name=exp_id, monitor='kappa_score'),
#             news_utils.fastai.SacredLogger(learn, ex),
#         ]

for i in range(1, 4):
    epochs = 1
    if i in [1, 2]:
        learn.freeze_to(-i)
    else:
        learn.unfreeze()
        epochs = 100
    learn.fit(epochs, np.array(lrs))

epoch,train_loss,valid_loss,accuracy,kappa_score,F1_bin,prec_bin,reca_bin
1,0.690971,0.664428,0.523546,0.183945,0.514124,0.883495,0.362550


epoch,train_loss,valid_loss,accuracy,kappa_score,F1_bin,prec_bin,reca_bin
1,0.655169,0.690048,0.650970,0.011561,0.212500,0.165049,0.298246


epoch,train_loss,valid_loss,accuracy,kappa_score,F1_bin,prec_bin,reca_bin
1,0.662588,0.656431,0.656510,0.071055,0.287356,0.242718,0.352113
2,0.647084,0.657620,0.700831,0.123235,0.280000,0.203883,0.446809
3,0.620392,0.668299,0.695291,0.137077,0.312500,0.242718,0.438596
4,0.644261,0.615450,0.731302,0.198567,0.331034,0.233010,0.571429
5,0.649279,0.592113,0.756233,0.248948,0.352941,0.233010,0.727273
6,0.629625,0.582902,0.728532,0.163601,0.279412,0.184466,0.575758
7,0.635249,0.565803,0.731302,0.187031,0.312057,0.213592,0.578947
8,0.607298,0.568865,0.756233,0.327775,0.476190,0.388350,0.615385
9,0.623233,0.563429,0.728532,0.187423,0.319444,0.223301,0.560976
10,0.604618,0.539045,0.764543,0.352815,0.497041,0.407767,0.636364
